In [1]:
import pandas as pd
import numpy as np
from haversine import haversine
import datetime
import os
import paramiko
import json
import zipcodes

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

os.getcwd()

'/home/jian/SunnyD/Waltmart_TA_20190620'

In [2]:
'''
import paramiko

safegraph_folder="/home/jian/SunnyD/Waltmart_TA_20190620/safegraph_0530/"
try:
    os.stat(safegraph_folder)
except:
    os.mkdir(safegraph_folder)
    
    
host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

remote_folder="/mnt/drv5/SafeGraph/placements_20190530/poi/"

remote_list=sftp.listdir(remote_folder)
remote_list=[remote_folder+ x for x in remote_list]

for remote_file in remote_list:
    if ".csv" in remote_file:
        basename=os.path.basename(remote_file)
        local_path=safegraph_folder+"POI_"+basename
        sftp.get(remote_file,local_path)

        
remote_brand_info="/mnt/drv5/SafeGraph/placements_20190530/brand_info/part-00000-tid-4126444816223203350-e75d9da9-6a49-4305-9df7-e17fcb021802-143-c000.csv.gz"
local_path=safegraph_folder+"brand_info_"+os.path.basename(remote_brand_info)
sftp.get(remote_brand_info,local_path)
sftp.close()
'''

'\nimport paramiko\n\nsafegraph_folder="/home/jian/SunnyD/Waltmart_TA_20190620/safegraph_0530/"\ntry:\n    os.stat(safegraph_folder)\nexcept:\n    os.mkdir(safegraph_folder)\n    \n    \nhost = "64.237.51.251" #hard-coded\nport = 22\ntransport = paramiko.Transport((host, port))\n\npassword = "jian@juba2017" #hard-coded\nusername = "jian" #hard-coded\ntransport.connect(username = username, password = password)\nsftp = paramiko.SFTPClient.from_transport(transport)\n\nremote_folder="/mnt/drv5/SafeGraph/placements_20190530/poi/"\n\nremote_list=sftp.listdir(remote_folder)\nremote_list=[remote_folder+ x for x in remote_list]\n\nfor remote_file in remote_list:\n    if ".csv" in remote_file:\n        basename=os.path.basename(remote_file)\n        local_path=safegraph_folder+"POI_"+basename\n        sftp.get(remote_file,local_path)\n\n        \nremote_brand_info="/mnt/drv5/SafeGraph/placements_20190530/brand_info/part-00000-tid-4126444816223203350-e75d9da9-6a49-4305-9df7-e17fcb021802-143-c000.

In [3]:
safegraph_folder="/home/jian/SunnyD/Waltmart_TA_20190620/safegraph_0530/"


In [4]:
df_brand_info=pd.read_table("/home/jian/SunnyD/Waltmart_TA_20190620/safegraph_0530/brand_info_part-00000-tid-4126444816223203350-e75d9da9-6a49-4305-9df7-e17fcb021802-143-c000.csv.gz",
                            dtype=str,sep="|")
walmart_store_name=[x for x in df_brand_info['brand_name'].unique().tolist() if "walmart" in x.lower()]

In [5]:
poi_file_list=os.listdir(safegraph_folder)
poi_file_list=[x for x in poi_file_list if "POI" in x]
df_walmart=pd.DataFrame()
for file in poi_file_list:
    file=safegraph_folder+file
    df=pd.read_table(file,dtype=str,sep="|",compression="gzip")
    df=df[df['brands'].isin(walmart_store_name)]
    df_walmart=df_walmart.append(df)

In [6]:
df_walmart_stores=df_walmart.loc[:,['safegraph_place_id','brands','location_name','street_address','city','state','zip_code','latitude','longitude']]
df_walmart_stores['latitude']=df_walmart_stores['latitude'].astype(float)
df_walmart_stores['longitude']=df_walmart_stores['longitude'].astype(float)
df_walmart_stores=df_walmart_stores.reset_index()

print(df_walmart_stores.shape)
print(df_walmart_stores['safegraph_place_id'].nunique())

del df_walmart_stores['index']

df_walmart_stores=df_walmart_stores.rename(columns={"city":"city_of_store","state":"state_of_store","zip_code":"zip_of_store"})


(4691, 10)
4691


In [7]:
df_walmart_stores.head(2)

,safegraph_place_id,brands,location_name,street_address,city_of_store,state_of_store,zip_of_store,latitude,longitude
0,sg:7a7b6c84862f4a7eb4b0c5807efc5fb9,Walmart,Walmart,80 town line road,rocky hill,ct,06067,41.678841,-72.657477
1,sg:f2e0eee3e1f047fd82c373ba6f3dd587,Walmart,Walmart,355 lincoln avenue,east stroudsburg,pa,18301,40.989522,-75.183115


In [8]:
i_counter=0

df_by_store_zip=pd.DataFrame()
for ind,row in df_walmart_stores.iterrows():
    store_center=[row['latitude'],row['longitude']]
    safegraph_place_id=row['safegraph_place_id']
    df_1_store=pd.DataFrame()
    for zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"safegraph_place_id":safegraph_place_id,"zip_cd":zip_cd,"dist_miles":dist},index=[0])
            df_1_store=df_1_store.append(df)
            
    i_counter+=1        
    df_by_store_zip=df_by_store_zip.append(df_1_store)
        
    if i_counter%100==10:
        print(i_counter,datetime.datetime.now())

10 2019-06-21 11:33:16.389377
110 2019-06-21 11:33:27.377034
210 2019-06-21 11:33:38.107732
310 2019-06-21 11:33:47.979854
410 2019-06-21 11:33:58.667785
510 2019-06-21 11:34:09.632748
610 2019-06-21 11:34:19.966558
710 2019-06-21 11:34:30.536139
810 2019-06-21 11:34:40.775072
910 2019-06-21 11:34:51.508819
1010 2019-06-21 11:35:01.847070
1110 2019-06-21 11:35:12.582872
1210 2019-06-21 11:35:22.699211
1310 2019-06-21 11:35:33.056161
1410 2019-06-21 11:35:43.593219
1510 2019-06-21 11:35:53.939836
1610 2019-06-21 11:36:04.446282
1710 2019-06-21 11:36:15.198259
1810 2019-06-21 11:36:25.851905
1910 2019-06-21 11:36:35.909737
2010 2019-06-21 11:36:46.812500
2110 2019-06-21 11:36:57.130342
2210 2019-06-21 11:37:07.495839
2310 2019-06-21 11:37:17.516294
2410 2019-06-21 11:37:27.832618
2510 2019-06-21 11:37:39.347654
2610 2019-06-21 11:37:49.201273
2710 2019-06-21 11:37:59.901778
2810 2019-06-21 11:38:11.074847
2910 2019-06-21 11:38:22.246679
3010 2019-06-21 11:38:32.835507
3110 2019-06-21 11:

In [9]:
output_zip_store=pd.merge(df_walmart_stores,df_by_store_zip,on="safegraph_place_id",how="left")
output_zip_store.shape

(97633, 11)

In [10]:
Nielsen_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2,6],skiprows=1)


Nielsen_DMA.columns=['zip_cd','DMA','State']
zip_state_dict=Nielsen_DMA[['zip_cd','State']].drop_duplicates()
zip_state_dict=zip_state_dict.groupby(['zip_cd'])['State'].apply(list).to_frame().reset_index()
zip_state_dict=zip_state_dict.set_index('zip_cd').to_dict()['State']

del Nielsen_DMA['State']
Nielsen_DMA=Nielsen_DMA.drop_duplicates()
Nielsen_DMA=Nielsen_DMA.groupby(['zip_cd'])['DMA'].apply(list).to_frame().reset_index()
Nielsen_DMA.head(3)

,zip_cd,DMA
0,00501,[NEW YORK]
1,00544,[NEW YORK]
2,01001,[SPRINGFIELD-HOLYOKE]


In [11]:
def add_city_of_zip(x):
    try:
        y=zipcodes.matching(x)[0]['city']
    except:
        y=np.nan
    return y

def add_state_of_zip(x):
    try:
        y=zip_state_dict[x]
    except:
        try:
            y=zipcodes.matching(x)[0]['state']
        except:
            y=np.nan
    return y


        

In [12]:
output_unique_zip=output_zip_store[['zip_cd','safegraph_place_id','dist_miles']].sort_values(['zip_cd','dist_miles'],ascending=[True,True])
output_unique_zip=output_unique_zip.drop_duplicates('zip_cd')
output_unique_zip=output_unique_zip.rename(columns={'safegraph_place_id':"nearest_store","dist_miles":"nearest_dist_miles"})


output_unique_zip['city_of_zip']=output_unique_zip['zip_cd'].apply(lambda x: add_city_of_zip(x))
output_unique_zip['state_of_zip']=output_unique_zip['zip_cd'].apply(lambda x: add_state_of_zip(x))

output_unique_zip=pd.merge(output_unique_zip,Nielsen_DMA,on="zip_cd",how="left")
output_unique_zip=output_unique_zip.rename(columns={"DMA":"DMA_of_zip"})

In [15]:
output_unique_zip.shape

(24048, 6)

In [16]:
output_unique_zip.head(43)

,zip_cd,nearest_store,nearest_dist_miles,city_of_zip,state_of_zip,DMA_of_zip
0,00501,sg:930a562e69ea477d8214520bb5d63fb8,3.656380,HOLTSVILLE,[NY],[NEW YORK]
1,01001,sg:b15739f3d1a24b939ed770860f068604,5.234710,AGAWAM,[MA],[SPRINGFIELD-HOLYOKE]
2,01002,sg:cdc4ca3af5974282ab8149c81331500e,2.422551,AMHERST,[MA],[SPRINGFIELD-HOLYOKE]
3,01003,sg:cdc4ca3af5974282ab8149c81331500e,2.747130,AMHERST,[MA],[SPRINGFIELD-HOLYOKE]
4,01004,sg:cdc4ca3af5974282ab8149c81331500e,2.305009,AMHERST,[MA],[SPRINGFIELD-HOLYOKE]
5,01007,sg:cdc4ca3af5974282ab8149c81331500e,8.247356,BELCHERTOWN,[MA],[SPRINGFIELD-HOLYOKE]
6,01009,sg:bcf87d8630c441ec8bf6602b1b9c3ba8,3.923703,BONDSVILLE,[MA],[SPRINGFIELD-HOLYOKE]
7,01010,sg:54830a2150fd4b388093776ed8114ce3,7.380549,BRIMFIELD,[MA],"[BOSTON (MANCHESTER), SPRINGFIELD-HOLYOKE]"
8,01012,sg:c88f8dc700614d40aea8480eaef57311,8.833332,CHESTERFIELD,[MA],[SPRINGFIELD-HOLYOKE]
9,01013,sg:8580188e0cd84c29bafed38290b30baa,1.768123,CHICOPEE,[MA],[SPRINGFIELD-HOLYOKE]


In [25]:
Nielsen_DMA[Nielsen_DMA['zip_cd'].isin(missing_zip_list)].shape

(17, 2)

In [26]:
len(missing_zip_list)

19